In [ ]:
from LSTM_model import LSTM
import numpy as np
import random
import torch


In [ ]:
# Setting random seed for reproducibility
torch.manual_seed(140)
np.random.seed(140)
random.seed(140)

In [ ]:
# Simulate data 
x_values = torch.tensor([1, 2, 3], dtype=torch.float32).view(-1, 1, 1)
y_values = torch.tensor([10, 20, 30], dtype=torch.float32).view(-1, 1, 1)

# Create an instance of the LSTM model
model = LSTM(x_values, y_values, input_size=1, hidden_size=10, num_layers=1, output_size=1, batch_size=3, num_epochs=50, learning_rate=0.01)
model.train_model() # Train the model

# Get a prediction
x_test = torch.tensor([4], dtype=torch.float32).view(-1, 1, 1)
x_test = model.transform(x_test)
y_pred = model(x_test)
y_pred_orig = model.inverse_transform_y(y_pred)

print(f'Prediction: {y_pred_orig.item()}')  # Prediction: 40.0

In [ ]:
def calculate_AARE_star(D, D_bar, T):
    if T < 5:
        raise ValueError("T must be greater than or equal to 5")
    
    # Calculate the absolute relative errors for the required time period
    absolute_relative_errors = torch.abs(D[T-3:T+1] - D_bar[T-3:T+1]) / D[T-3:T+1]
    
    # Calculate the average absolute relative error
    AARE_star = torch.mean(absolute_relative_errors)
    
    return AARE_star

# Example usage:
# D = torch.tensor([...]) # Tensor containing D values
# D_bar = torch.tensor([...]) # Tensor containing D_bar values
# T = current time period (must be >= 5)
# AARE_star_T = calculate_AARE_star(D, D_bar, T)



In [ ]:
def calculate_mu_AARE_star(AARE_star, T, W):
    if 7 <= T < W + 4:
        mu_AARE_star = torch.mean(AARE_star[4:T])
    elif T >= W + 4:
        mu_AARE_star = torch.mean(AARE_star[T-W:T])
    else:
        raise ValueError("T must be greater than or equal to 7")
    return mu_AARE_star

def calculate_sigma_star(AARE_star, mu_AARE_star, T, W):
    if 7 <= T < W + 4:
        sigma_star = torch.sqrt(torch.sum((AARE_star[4:T] - mu_AARE_star) ** 2) / (T - 4))
    elif T >= W + 4:
        sigma_star = torch.sqrt(torch.sum((AARE_star[T-W:T] - mu_AARE_star) ** 2) / W)
    else:
        raise ValueError("T must be greater than or equal to 7")
    return sigma_star

def calculate_Thd_star(AARE_star, T, W):
    mu_AARE_star = calculate_mu_AARE_star(AARE_star, T, W)
    sigma_star = calculate_sigma_star(AARE_star, mu_AARE_star, T, W)
    Thd_star = mu_AARE_star + 3 * sigma_star
    return Thd_star

# Example usage:
# AARE_star = torch.tensor([...]) # A PyTorch tensor containing AARE* values
# T = current time period (must be >= 7)
# W = window size for the moving average and standard deviation
# Thd_star = calculate_Thd_star(AARE_star, T, W)

In [ ]:

# Let T be the current time point and T starts from 0; Let flag* be True;

While time has advanced {
    Collect data point D_T
    
    if T >= 2 && T < 5 {
        Train LSTM model by taking D_T-2, D_T-1 and D_T as training data (X and Y values for the three datapoints)
        Let M* be the resulting model and use M* to predict D_T+1
   
    } elif T >= 5 && T < 7 {
        Calculate AARE_T* based on Equation 5
        Train LSTM model by taking D_T-2, D_T-1 and D_T as training data (X and Y values for the three datapoints)
        Let M* be the resulting model and use M* to predict D_T+1
        
    } elif T >= 7 && flag* == True {
        if T != 7 {Use M* to Predict D_T}
        Calculate AARE_T* based on Equation 5
        Calculate Thd* based on Equation 6

        if AARE_T* <= Thd* {D_T* is not considered an anomaly}

        else {
            Train LSTM model by taking D_T-3, D_T-2 and D_T-1 as training data
            Use the model to repredict D_T
            Re-calculate AARE_T* based on Equation 5
            Re-calculate Thd* based on Equation 6

            if AARE_T* <= Thd* {
                D_T is not consideren an anomaly
                Replace M* with the new LSTM-model
                Let flag* be True

            } else {
                D_T is reported as an anomaly immediately
                Let flag* be True

            }
        }
    } elif T >= 7 && flag* == False {
        Train LSTM model by taking D_T-3, D_T-2 and D_T-1 as training data
        Use the LSTM model to predict D_T 
        Calculate AARE_T* based on Equation 5
        Calculate Thd* based on Equation 6

        if AARE_T* <= Thd* {
            D_T is not considered an anomaly
            Replace M* with the new LSTM-model
            Let flag* be True
        } else {
            D_T is reported as an anomaly immediately
            Let flag* be False
        }
    }


}

